# Teoria współbieżności

## Algorytm Eliminacji Gaussa z elementami współbieżności

### Maciej Wiśniewski grupa 9 środa 8.00

##### Cel zadania
Zadanie polegało na analizie algorytmu eliminacji Gaussa w kontekście teorii współbieżności oraz zaprojektowaniu jego współbieżnej implementacji. W tym celu zidentyfikowano niepodzielne czynności algorytmu, zbudowano alfabet w sensie teorii śladów, określono relacje zależności, a następnie wyznaczono graf zależności Diekerta oraz postać normalną Foaty. Na tej podstawie zaimplementowano współbieżny algorytm eliminacji Gaussa, działający poprawnie dla macierzy o dowolnym rozmiarze N.

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Rectangle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import threading
from IPython.display import display, HTML, Markdown
import warnings
import copy
from typing import List, Dict, Tuple

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
np.set_printoptions(precision=3, suppress=True)

Dodamy metody do obługi plików

In [2]:
def process_file(input_file):
    if isinstance(input_file, str):
        with open(input_file, 'r', encoding='utf8') as f: text = f.read()
    else:
        text = input_file.read()
    
    lines = text.strip().split("\n")
    N = int(lines[0])
    
    matrix = [[None for _ in range(N)] for _ in range(N)]
    column = [[None] for _ in range(N)]
    
    for i in range(N):
        row_elems = lines[i + 1].split()
        for j in range(N):
            matrix[i][j] = float(row_elems[j])

    column_elems = lines[N + 1].split()
    for i in range(N): column[i][0] = float(column_elems[i])
    
    return N, matrix, column


def load_data_from_file(filename):
    try:
        N, matrix, column = process_file(filename)
        complemented_matrix = [row + col for row, col in zip(matrix, column)]
        
        return N, matrix, column, complemented_matrix  
    except FileNotFoundError:
        return None, None, None, None
    except Exception as e:
        return None, None, None, None

Przykładowe wywołanie

In [3]:
N, matrix, column, complemented_matrix = load_data_from_file('in.txt')

complemented_matrix = [row + col for row, col in zip(matrix, column)]

print("Macierz początkowa [A|b]:")
for i, row in enumerate(complemented_matrix):
    print(f"  [{' '.join([f'{x:7.2f}' for x in row])}]")

print(f"\nRozmiar: {N}x{N}")
print(f"Układ równań: Ax = b")

print("\nUkład równań:")
for i in range(N):
    eq = " + ".join([f"{matrix[i][j]:.1f}·x_{j+1}" for j in range(N)])
    print(f"  {eq} = {column[i][0]:.1f}")

Macierz początkowa [A|b]:
  [   2.00    1.00    3.00    6.00]
  [   4.00    3.00    8.00   15.00]
  [   6.00    5.00   16.00   27.00]

Rozmiar: 3x3
Układ równań: Ax = b

Układ równań:
  2.0·x_1 + 1.0·x_2 + 3.0·x_3 = 6.0
  4.0·x_1 + 3.0·x_2 + 8.0·x_3 = 15.0
  6.0·x_1 + 5.0·x_2 + 16.0·x_3 = 27.0


##### Współbieżna eliminacja Gaussa

W algorytmie współbieżnej eliminacji Gaussa wyróżniono trzy podstawowe typy niepodzielnych zadań obliczeniowych, oznaczone jako **A**, **B** oraz **C**, które stanowią podstawę dalszej analizy współbieżności algorytmu.

- **Aᵢ,ₖ** – zadanie polegające na wyznaczeniu mnożnika wykorzystywanego do eliminacji elementów znajdujących się pod elementem głównym macierzy. Mnożnik ten obliczany jest zgodnie ze wzorem:  
  *mₖ,ᵢ = Mₖ,ᵢ / Mᵢ,ᵢ*.

- **Bᵢ,ⱼ,ₖ** – zadanie polegające na pomnożeniu *j*-tego elementu wiersza *i* przez wcześniej obliczony mnożnik. Wynik tej operacji stanowi wartość pośrednią używaną w procesie eliminacji:  
  *nₖ,ᵢ = Mᵢ,ⱼ · mₖ,ᵢ*.

- **Cᵢ,ⱼ,ₖ** – zadanie realizujące właściwy krok eliminacji Gaussa, polegające na odjęciu obliczonej wartości pośredniej od odpowiedniego elementu wiersza *k*, zgodnie ze wzorem:  
  *Mₖ,ⱼ = Mₖ,ⱼ − nₖ,ᵢ*.

Powyższe zadania stanowią niepodzielne operacje obliczeniowe, na których oparto dalszą analizę relacji zależności oraz konstrukcję grafu współbieżności.

Na początku analizy zdefiniujemy klasę reprezentująca operacje


In [4]:
class GaussOperation: 
    def __init__(self):
        self.mki = {}  # mnożniki
        self.nki = {}  # pomnożone wartości
        self.history = []  # historia operacji
    
    def operation_A(self, M, i, k):
        """Oblicz mnożnik: m_ki = M[k,i] / M[i,i]"""
        key = f"{k}_{i}"
        self.mki[key] = M[k-1][i-1] / M[i-1][i-1]
        self.history.append(f"A({i},{k}): m_{{{k},{i}}} = {self.mki[key]:.3f}")
        return self.mki[key]
    
    def operation_B(self, M, i, j, k):
        """Pomnóż element: n_kij = M[i,j] * m_ki"""
        key_m = f"{k}_{i}"
        key_n = f"{k}_{i}_{j}"
        self.nki[key_n] = M[i-1][j-1] * self.mki[key_m]
        self.history.append(f"B({i},{j},{k}): n_{{{k},{i},{j}}} = {self.nki[key_n]:.3f}")
        return self.nki[key_n]
    
    def operation_C(self, M, i, j, k):
        """Odejmij: M[k,j] -= n_kij"""
        key = f"{k}_{i}_{j}"
        old_val = M[k-1][j-1]
        M[k-1][j-1] -= self.nki[key]
        self.history.append(f"C({i},{j},{k}): M[{k},{j}] = {old_val:.3f} - {self.nki[key]:.3f} = {M[k-1][j-1]:.3f}")


Następnie stworzymy metody do obsługi tych operacji i alfabetu

In [5]:
def get_alphabet(N):
    alphabet = []
    line_breaks = []
    
    for i in range(1, N):
        for k in range(i + 1, N + 1):
            alphabet.append({"oper": "A", "i": i, "k": k})
            for j in range(i, N + 2):
                alphabet.append({"oper": "B", "i": i, "j": j, "k": k})
                alphabet.append({"oper": "C", "i": i, "j": j, "k": k})
            
            if i < N - 1:
                if line_breaks: line_breaks.append(line_breaks[-1] + 2 * (N + 2 - i) + 1)
                else: line_breaks.append(2 * (N + 2 - i))
    
    return alphabet, line_breaks

def print_op(op): return f"A_({op['i']},{op['k']})" if op["oper"] == "A" else f"{op['oper']}_({op['i']},{op['j']},{op['k']})"

Potrzebujemy tez funckji do obliczania relacji zaleznosci i Postaci Normalnej Foaty FNF

In [6]:
def get_dependent_transactions(N):
    dependencies = []
    breaks = []

    def add_break(value):
        if breaks: breaks.append(breaks[-1] + value)
        else: breaks.append(value)

    for i in range(1, N):
        for k in range(i + 1, N + 1):
            opA = {"oper": "A", "i": i, "k": k}

            for j in range(i, N + 2):
                opB = {"oper": "B", "i": i, "j": j, "k": k}
                dependencies.append((opA, opB))
            add_break(N + 1 - i)
            for j in range(i, N + 2):
                opB = {"oper": "B", "i": i, "j": j, "k": k}
                opC = {"oper": "C", "i": i, "j": j, "k": k}
                dependencies.append((opB, opC))
            add_break(N + 2 - i)

    for i in range(2, N):
        for k in range(i + 1, N + 1):
            opA = {"oper": "A", "i": i, "k": k}
            dependencies.append(({"oper": "C", "i": i - 1, "j": i, "k": i}, opA))
            dependencies.append(({"oper": "C", "i": i - 1, "j": i, "k": k}, opA))

        add_break(N + 1 - i)
        for k in range(i + 1, N + 1):
            for j in range(i + 1, N + 2):
                dependencies.append(
                    ({"oper": "C", "i": i - 1, "j": j, "k": i},
                     {"oper": "B", "i": i, "j": j, "k": k}))

        add_break(N + 1 - i)

        for k in range(i + 1, N + 1):
            for j in range(i + 1, N + 2):
                dependencies.append(
                    ({"oper": "C", "i": i - 1, "j": j, "k": k},
                     {"oper": "C", "i": i, "j": j, "k": k}))

        if i < N - 1: add_break(N + 1 - i)

    return dependencies, breaks


In [7]:
def get_fnf(N):
    fnf = []
    
    for i in range(1, N):
        fnf.append([])
        for k in range(i + 1, N + 1): fnf[-1].append({"oper": "A", "i": i, "k": k})
        fnf.append([])
        for k in range(i + 1, N + 1):
            for j in range(i, N + 2):
                fnf[-1].append({"oper": "B", "i": i, "j": j, "k": k})
        fnf.append([])
        for k in range(i + 1, N + 1):
            for j in range(i, N + 2):
                fnf[-1].append({"oper": "C", "i": i, "j": j, "k": k})
    
    return fnf

Algorytm eliminacji Gaussa

In [8]:
def backward_substitution(eliminated_matrix, N):
    results = [0 for _ in range(N)]
    
    for i in range(N - 1, -1, -1):
        for j in range(i + 1, N):
            eliminated_matrix[i][N] -= eliminated_matrix[i][j] * results[j]
            eliminated_matrix[i][j] = 0.
        eliminated_matrix[i][N] /= eliminated_matrix[i][i]
        results[i] = eliminated_matrix[i][N]
        eliminated_matrix[i][i] = 1.
    
    return results

In [9]:
def parallel_gauss(matrix, fnf):
    M = [[el for el in row] for row in matrix]
    ops = GaussOperation()
    
    for foata_class in fnf:
        thread_list = []
        for task in foata_class:
            if task["oper"] == "A":
                thread = threading.Thread(target=ops.operation_A, args=(M, task["i"], task["k"]))
            elif task["oper"] == "B":
                thread = threading.Thread(target=ops.operation_B, args=(M, task["i"], task["j"], task["k"]))
            else: 
                thread = threading.Thread(target=ops.operation_C,args=(M, task["i"], task["j"], task["k"]))
            
            thread_list.append(thread)
        
        for thread in thread_list: thread.start()
        for thread in thread_list: thread.join()
    
    return M, ops

Mając zdefiniowane wszystkie metody możemy przejść do działania. Dla przykładu użyjemy danych z pliku *in.txt*

In [13]:
N, matrix, column, complemented_matrix = load_data_from_file('in.txt')

alphabet, line_breaks = get_alphabet(N)
print("Σ = {", end="")
for i, op in enumerate(alphabet):
    print(print_op(op), end="")
    if i < len(alphabet) - 1: print(", ", end="")
    if i in line_breaks:  print("\n     ", end="")

Σ = {A_(1,2), B_(1,1,2), C_(1,1,2), B_(1,2,2), C_(1,2,2), B_(1,3,2), C_(1,3,2), B_(1,4,2), C_(1,4,2), 
     A_(1,3), B_(1,1,3), C_(1,1,3), B_(1,2,3), C_(1,2,3), B_(1,3,3), C_(1,3,3), B_(1,4,3), C_(1,4,3), 
     A_(2,3), B_(2,2,3), C_(2,2,3), B_(2,3,3), C_(2,3,3), B_(2,4,3), C_(2,4,3)

In [ ]:


################################################################################
# Komórka 9: Relacja Zależności
################################################################################

dependent, dep_line_breaks = get_dependent_transactions(N)

print(f"RELACJA ZALEŻNOŚCI D")
print("=" * 60)
print(f"Liczba par zależnych: {len(dependent)}\n")

print("D = sym{", end="")
for i, (dep1, dep2) in enumerate(dependent):
    print(f"({print_op(dep1)},{print_op(dep2)})", end="")
    if i < len(dependent) - 1:
        print(", ", end="")
        if i in dep_line_breaks:
            print("\n        ", end="")
print("}⁺ ∪ I_Σ\n")

# Analiza typów zależności
dep_types = {}
for dep1, dep2 in dependent:
    key = f"{dep1['oper']}-{dep2['oper']}"
    dep_types[key] = dep_types.get(key, 0) + 1

print("Typy zależności:")
for dep_type, count in sorted(dep_types.items()):
    print(f"  {dep_type}: {count}")

################################################################################
# Komórka 10: Postać Normalna Foaty
################################################################################

fnf = get_fnf(N)

print(f"POSTAĆ NORMALNA FOATY")
print("=" * 60)
print(f"Liczba klas Foaty: {len(fnf)}\n")

print("FNF([w]) = ", end="")
for i, foata_class in enumerate(fnf):
    print("[", end="")
    for op in foata_class:
        print(print_op(op), end="")
    print("]", end="")
    if i % 2 == 1 and i < len(fnf) - 1:
        print("\n           ", end="")
print("\n")

# Szczegółowa analiza klas
print("Szczegóły klas Foaty:")
for i, foata_class in enumerate(fnf):
    op_type = foata_class[0]["oper"] if foata_class else "?"
    row_num = foata_class[0]["i"] if foata_class else "?"
    print(f"  Klasa {i+1}: {len(foata_class):3d} operacji typu {op_type} dla wiersza {row_num}")

# Potencjał równoległości
max_parallel = max(len(fc) for fc in fnf)
print(f"\nMaksymalne równoległość: {max_parallel} operacji jednocześnie")
print(f"Liczba kroków sekwencyjnych: {len(fnf)}")

################################################################################
# Komórka 11: Wizualizacja - Graf Zależności Diekerta
################################################################################

def visualize_diekert_graph(dependent, fnf, figsize=(16, 10)):
    """Rysuje graf zależności Diekerta z kolorowaniem według klas Foaty"""
    
    G = nx.DiGraph()
    node_colors_palette = ["#00FF00", "#FFA500", "#00FFFF", "#FF69B4", 
                          "#FFD700", "#FF6347", "#98FB98", "#DDA0DD"]
    
    max_len = max(len(fc) for fc in fnf)
    
    # Dodawanie wierzchołków
    positions = {}
    colors = []
    
    for i, foata_class in enumerate(fnf):
        shift_pos = max_len / len(foata_class) if len(foata_class) > 0 else 1
        base_pos = shift_pos / 2
        
        for j, op in enumerate(foata_class):
            node = print_op(op)
            G.add_node(node)
            positions[node] = [base_pos + j * shift_pos, len(fnf) - i]
            colors.append(node_colors_palette[i % len(node_colors_palette)])
    
    # Dodawanie krawędzi
    for dep1, dep2 in dependent:
        node1 = print_op(dep1)
        node2 = print_op(dep2)
        if node1 in G.nodes() and node2 in G.nodes():
            G.add_edge(node1, node2)
    
    # Rysowanie
    plt.figure(figsize=figsize)
    nx.draw(G, positions, node_color=colors, node_size=1200, 
            with_labels=True, font_size=7, font_weight='bold',
            arrows=True, arrowsize=10, arrowstyle='->', 
            edge_color='gray', width=1.5, alpha=0.7)
    
    plt.title("Graf Zależności Diekerta z kolorowaniem według klas Foaty", 
              fontsize=16, fontweight='bold', pad=20)
    
    # Legenda
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=node_colors_palette[i % len(node_colors_palette)], 
                             label=f'Klasa Foaty {i+1}') 
                      for i in range(len(fnf))]
    plt.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1))
    
    plt.tight_layout()
    plt.show()
    
    print(f"Graf zawiera:")
    print(f"  - Wierzchołki: {G.number_of_nodes()}")
    print(f"  - Krawędzie: {G.number_of_edges()}")

# Rysowanie grafu
visualize_diekert_graph(dependent, fnf)

################################################################################
# Komórka 12: Alternatywna Wizualizacja - Graf Warstwowy
################################################################################

def visualize_layered_graph(fnf, figsize=(14, 8)):
    """Wizualizacja grafu w formie warstwowej - każda klasa Foaty to warstwa"""
    
    fig, ax = plt.subplots(figsize=figsize)
    
    colors_palette = ["#00FF00", "#FFA500", "#00FFFF", "#FF69B4", 
                     "#FFD700", "#FF6347", "#98FB98", "#DDA0DD"]
    
    layer_height = 1.5
    node_radius = 0.15
    
    for layer_idx, foata_class in enumerate(fnf):
        y = len(fnf) - layer_idx
        num_ops = len(foata_class)
        
        if num_ops == 0:
            continue
            
        spacing = 10 / num_ops if num_ops > 0 else 1
        start_x = (10 - spacing * (num_ops - 1)) / 2
        
        for i, op in enumerate(foata_class):
            x = start_x + i * spacing
            
            # Rysowanie węzła
            circle = plt.Circle((x, y), node_radius, 
                              color=colors_palette[layer_idx % len(colors_palette)],
                              ec='black', linewidth=2, zorder=3)
            ax.add_patch(circle)
            
            # Etykieta
            ax.text(x, y, print_op(op), ha='center', va='center', 
                   fontsize=6, fontweight='bold', zorder=4)
        
        # Linia warstwy
        ax.plot([0, 10], [y, y], 'k--', alpha=0.3, linewidth=1, zorder=1)
        ax.text(-0.5, y, f'Warstwa {layer_idx + 1}', ha='right', va='center',
               fontsize=10, fontweight='bold')
    
    ax.set_xlim(-1, 11)
    ax.set_ylim(0, len(fnf) + 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Graf Warstwowy - Klasy Foaty', fontsize=16, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()

visualize_layered_graph(fnf)

################################################################################
# Komórka 13: Wykonanie Eliminacji Gaussa
################################################################################

print("WYKONYWANIE ELIMINACJI GAUSSA")
print("=" * 60)

# Kopia macierzy do obliczeń
complemented_copy = copy.deepcopy(complemented_matrix)

print("Macierz początkowa:")
for row in complemented_copy:
    print([f"{x:8.3f}" for x in row])

# Wykonanie eliminacji
eliminated_matrix, ops = parallel_gauss(complemented_copy, fnf)

print(f"\nMacierz po eliminacji:")
for row in eliminated_matrix:
    print([f"{x:8.3f}" for x in row])

print(f"\nWykonano {len(ops.history)} operacji elementarnych")

################################################################################
# Komórka 14: Podstawianie Wsteczne i Rozwiązanie
################################################################################

print("PODSTAWIANIE WSTECZNE")
print("=" * 60)

# Kopia macierzy do podstawiania wstecznego
result_matrix = copy.deepcopy(eliminated_matrix)

# Obliczenie rozwiązania
results = backward_substitution(result_matrix, N)

print("Macierz po podstawianiu wstecznym (postać jednostkowa):")
for row in result_matrix:
    print([f"{x:8.3f}" for x in row])

print(f"\nWEKTOR ROZWIĄZAŃ:")
print("=" * 60)
for i, val in enumerate(results):
    print(f"x_{i+1} = {val:8.3f}")

################################################################################
# Komórka 15: Weryfikacja Rozwiązania
################################################################################

print("WERYFIKACJA ROZWIĄZANIA")
print("=" * 60)

# Sprawdzenie: A * x = b
verification = []
for i in range(N):
    sum_val = sum(matrix[i][j] * results[j] for j in range(N))
    verification.append(sum_val)
    expected = column[i][0]
    diff = abs(sum_val - expected)
    
    print(f"Równanie {i+1}: ", end="")
    print(" + ".join([f"{matrix[i][j]:.2f}*{results[j]:.3f}" for j in range(N)]), end="")
    print(f" = {sum_val:.3f} (oczekiwano: {expected:.3f}, błąd: {diff:.6f})")

max_error = max(abs(verification[i] - column[i][0]) for i in range(N))
print(f"\nMaksymalny błąd: {max_error:.10f}")

if max_error < 1e-6:
    print("✓ Rozwiązanie POPRAWNE!")
else:
    print("⚠ Rozwiązanie zawiera błędy")

################################################################################
# Komórka 16: Wizualizacja Macierzy - Heatmapa
################################################################################

def visualize_matrix_evolution(original, eliminated, final, results):
    """Wizualizuje ewolucję macierzy w trakcie eliminacji"""
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    matrices = [original, eliminated, final]
    titles = ['Macierz Początkowa [A|b]', 
              'Po Eliminacji Gaussa', 
              'Po Podstawianiu Wstecznym [I|x]']
    
    for idx, (ax, mat, title) in enumerate(zip(axes, matrices, titles)):
        mat_array = np.array(mat)
        im = ax.imshow(mat_array, cmap='RdYlGn', aspect='auto')
        ax.set_title(title, fontsize=12, fontweight='bold')
        
        # Dodanie wartości w komórkach
        for i in range(len(mat)):
            for j in range(len(mat[0])):
                text = ax.text(j, i, f'{mat[i][j]:.2f}',
                             ha="center", va="center", color="black", 
                             fontsize=10, fontweight='bold')
        
        ax.set_xticks(range(len(mat[0])))
        ax.set_yticks(range(len(mat)))
        ax.set_xticklabels([f'col{j+1}' for j in range(len(mat[0]))], fontsize=9)
        ax.set_yticklabels([f'row{i+1}' for i in range(len(mat))], fontsize=9)
        
        plt.colorbar(im, ax=ax)
    
    plt.tight_layout()
    plt.show()

# Przygotowanie danych do wizualizacji
visualize_matrix_evolution(
    complemented_matrix,
    eliminated_matrix,
    result_matrix,
    results
)

################################################################################
# Komórka 17: Analiza Wydajności - Porównanie Sekwencyjne vs Równoległe
################################################################################

def analyze_complexity(N):
    """Analiza złożoności obliczeniowej"""
    
    # Liczba operacji
    alphabet, _ = get_alphabet(N)
    fnf = get_fnf(N)
    
    total_ops = len(alphabet)
    sequential_steps = len(fnf)
    max_parallel = max(len(fc) for fc in fnf) if fnf else 0
    
    # Teoretyczna złożoność
    classic_complexity = (N**3) / 3  # Klasyczna eliminacja Gaussa
    
    print("ANALIZA WYDAJNOŚCI")
    print("=" * 60)
    print(f"Rozmiar macierzy: {N}x{N}\n")
    
    print("Liczba operacji:")
    print(f"  - Całkowita liczba operacji:        {total_ops}")
    print(f"  - Klasyczna złożoność O(n³/3):      {classic_complexity:.0f}\n")
    
    print("Wykonanie równoległe (FNF):")
    print(f"  - Liczba kroków sekwencyjnych:      {sequential_steps}")
    print(f"  - Maksymalne równoległość:          {max_parallel}")
    print(f"  - Średnia operacji na krok:         {total_ops/sequential_steps:.1f}\n")
    
    speedup = total_ops / sequential_steps if sequential_steps > 0 else 0
    efficiency = speedup / max_parallel if max_parallel > 0 else 0
    
    print("Metryki wydajności:")
    print(f"  - Teoretyczne przyspieszenie:       {speedup:.2f}x")
    print(f"  - Efektywność wykorzystania:        {efficiency*100:.1f}%")
    
    return {
        'total_ops': total_ops,
        'sequential_steps': sequential_steps,
        'max_parallel': max_parallel,
        'speedup': speedup
    }

stats = analyze_complexity(N)


In [ ]:
################################################################################
# Komórka 18: Porównanie dla Różnych Rozmiarów Macierzy
################################################################################

def compare_sizes(sizes=[2, 3, 4, 5, 6]):
    """Porównanie wydajności dla różnych rozmiarów"""
    
    results = []
    for n in sizes:
        alphabet, _ = get_alphabet(n)
        fnf = get_fnf(n)
        
        results.append({
            'N': n,
            'total_ops': len(alphabet),
            'steps': len(fnf),
            'max_parallel': max(len(fc) for fc in fnf) if fnf else 0
        })
    
    # Tabela wyników
    print("PORÓWNANIE DLA RÓŻNYCH ROZMIARÓW")
    print("=" * 80)
    print(f"{'N':>3} | {'Operacje':>10} | {'Kroki':>6} | {'Max równ.':>10} | {'Przyspieszenie':>15}")
    print("-" * 80)
    
    for r in results:
        speedup = r['total_ops'] / r['steps'] if r['steps'] > 0 else 0
        print(f"{r['N']:>3} | {r['total_ops']:>10} | {r['steps']:>6} | {r['max_parallel']:>10} | {speedup:>14.2f}x")
    
    # Wykres
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    Ns = [r['N'] for r in results]
    ops = [r['total_ops'] for r in results]
    steps = [r['steps'] for r in results]
    
    ax1.plot(Ns, ops, 'o-', label='Całkowite operacje', linewidth=2, markersize=8)
    ax1.plot(Ns, steps, 's-', label='Kroki sekwencyjne', linewidth=2, markersize=8)
    ax1.set_xlabel('Rozmiar macierzy N', fontsize=12)
    ax1.set_ylabel('Liczba', fontsize=12)
    ax1.set_title('Liczba operacji vs kroki sekwencyjne', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    speedups = [r['total_ops']/r['steps'] if r['steps'] > 0 else 0 for r in results]
    ax2.plot(Ns, speedups, 'o-', color='green', linewidth=2, markersize=8)
    ax2.set_xlabel('Rozmiar macierzy N', fontsize=12)
    ax2.set_ylabel('Przyspieszenie', fontsize=12)
    ax2.set_title('Teoretyczne przyspieszenie', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

compare_sizes()

################################################################################
# Komórka 19: Interaktywna Wizualizacja z Plotly
################################################################################

def interactive_fnf_visualization(fnf):
    """Interaktywna wizualizacja FNF z użyciem Plotly"""
    
    fig = go.Figure()
    
    colors = ['rgb(0,255,0)', 'rgb(255,165,0)', 'rgb(0,255,255)', 
              'rgb(255,105,180)', 'rgb(255,215,0)', 'rgb(255,99,71)']
    
    for layer_idx, foata_class in enumerate(fnf):
        y_pos = len(fnf) - layer_idx
        num_ops = len(foata_class)
        
        if num_ops == 0:
            continue
        
        x_positions = np.linspace(0, 10, num_ops)
        
        for i, (x, op) in enumerate(zip(x_positions, foata_class)):
            fig.add_trace(go.Scatter(
                x=[x],
                y=[y_pos],
                mode='markers+text',
                marker=dict(
                    size=20,
                    color=colors[layer_idx % len(colors)],
                    line=dict(color='black', width=2)
                ),
                text=print_op(op),
                textposition='middle center',
                textfont=dict(size=8, color='black'),
                name=f'Klasa {layer_idx + 1}',
                showlegend=(i == 0),
                hovertemplate=f'<b>{print_op(op)}</b><br>Klasa Foaty: {layer_idx + 1}<extra></extra>'
            ))
    
    fig.update_layout(
        title='Interaktywna Wizualizacja Postaci Normalnej Foaty',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=True, zeroline=False, title='Klasa Foaty'),
        hovermode='closest',
        width=1000,
        height=600
    )
    
    fig.show()

# Wymaga zainstalowanego plotly
try:
    interactive_fnf_visualization(fnf)
    print("✓ Wizualizacja interaktywna wyświetlona")
except Exception as e:
    print("ℹ Zainstaluj plotly aby zobaczyć interaktywną wizualizację: pip install plotly")
    print(f"  Błąd: {e}")

################################################################################
# Komórka 20: Wizualizacja Wykonania Krok po Kroku
################################################################################

def visualize_execution_timeline(fnf):
    """Wizualizuje timeline wykonania z podziałem na kroki"""
    
    fig, ax = plt.subplots(figsize=(16, 8))
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(fnf)))
    
    y_position = 0
    timeline_data = []
    
    for step_idx, foata_class in enumerate(fnf):
        num_ops = len(foata_class)
        
        for op_idx, op in enumerate(foata_class):
            # Każda operacja jako prostokąt
            rect = Rectangle((step_idx, y_position), 0.8, 0.8,
                           facecolor=colors[step_idx],
                           edgecolor='black', linewidth=1.5)
            ax.add_patch(rect)
            
            # Etykieta operacji
            ax.text(step_idx + 0.4, y_position + 0.4, print_op(op),
                   ha='center', va='center', fontsize=7, fontweight='bold')
            
            timeline_data.append({
                'step': step_idx + 1,
                'operation': print_op(op),
                'y_pos': y_position
            })
            
            y_position += 1
        
        # Separator między krokami
        if step_idx < len(fnf) - 1:
            ax.axvline(x=step_idx + 0.9, color='red', linestyle='--', 
                      linewidth=2, alpha=0.7)
            y_position += 0.5
    
    # Etykiety kroków
    for step_idx in range(len(fnf)):
        ax.text(step_idx + 0.4, -1, f'Krok {step_idx + 1}',
               ha='center', va='top', fontsize=11, fontweight='bold')
    
    ax.set_xlim(-0.5, len(fnf))
    ax.set_ylim(-2, y_position + 1)
    ax.set_xlabel('Kroki czasowe (synchronizacja wątków)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Operacje równoległe', fontsize=12, fontweight='bold')
    ax.set_title('Timeline wykonania równoległego - każdy krok to synchronizacja wątków',
                fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nTimeline execution summary:")
    print(f"  - Całkowity czas (kroki): {len(fnf)}")
    print(f"  - Całkowita liczba operacji: {sum(len(fc) for fc in fnf)}")
    print(f"  - Średnio operacji na krok: {sum(len(fc) for fc in fnf) / len(fnf):.1f}")

visualize_execution_timeline(fnf)

################################################################################
# Komórka 21: Heatmapa Zależności Między Operacjami
################################################################################

def visualize_dependency_heatmap(dependent, alphabet):
    """Tworzy heatmapę pokazującą zależności między operacjami"""
    
    # Mapowanie operacji na indeksy
    op_to_idx = {print_op(op): i for i, op in enumerate(alphabet)}
    n = len(alphabet)
    
    # Macierz zależności
    dep_matrix = np.zeros((n, n))
    
    for dep1, dep2 in dependent:
        idx1 = op_to_idx.get(print_op(dep1))
        idx2 = op_to_idx.get(print_op(dep2))
        if idx1 is not None and idx2 is not None:
            dep_matrix[idx1][idx2] = 1
            dep_matrix[idx2][idx1] = 1  # Symetryczna
    
    # Wizualizacja
    fig, ax = plt.subplots(figsize=(12, 10))
    
    im = ax.imshow(dep_matrix, cmap='RdYlGn_r', aspect='auto', interpolation='nearest')
    
    # Dodanie linii siatki
    ax.set_xticks(np.arange(n))
    ax.set_yticks(np.arange(n))
    ax.set_xticklabels([print_op(op) for op in alphabet], rotation=90, fontsize=6)
    ax.set_yticklabels([print_op(op) for op in alphabet], fontsize=6)
    
    ax.set_title('Heatmapa Zależności Między Operacjami\n(1 = zależne, 0 = niezależne)',
                fontsize=14, fontweight='bold', pad=20)
    
    plt.colorbar(im, ax=ax, label='Zależność')
    plt.tight_layout()
    plt.show()
    
    # Statystyki
    total_pairs = n * (n - 1) / 2
    dependent_pairs = np.sum(dep_matrix) / 2
    independence_ratio = 1 - (dependent_pairs / total_pairs)
    
    print(f"\nStatystyki zależności:")
    print(f"  - Liczba operacji: {n}")
    print(f"  - Możliwych par: {int(total_pairs)}")
    print(f"  - Par zależnych: {int(dependent_pairs)}")
    print(f"  - Współczynnik niezależności: {independence_ratio*100:.1f}%")

visualize_dependency_heatmap(dependent, alphabet)

################################################################################
# Komórka 22: Analiza Struktur Danych
################################################################################

def analyze_data_structures():
    """Analiza struktur danych używanych w algorytmie"""
    
    print("ANALIZA STRUKTUR DANYCH")
    print("=" * 80)
    
    print("\n1. SŁOWNIKI POMOCNICZE:")
    print("   a) mki - mnożniki (m_k,i)")
    print("      - Klucz: 'k_i' (np. '2_1')")
    print("      - Wartość: float (m_k,i = M[k,i] / M[i,i])")
    print("      - Rozmiar dla NxN: O(N²)")
    
    print("\n   b) nki - wartości pomnożone (n_k,i,j)")
    print("      - Klucz: 'k_i_j' (np. '2_1_3')")
    print("      - Wartość: float (n_k,i,j = M[i,j] * m_k,i)")
    print("      - Rozmiar dla NxN: O(N³)")
    
    print("\n2. OPERACJE (ALFABET Σ):")
    print("   - Typ: Lista słowników")
    print("   - Struktura słownika: {'oper': str, 'i': int, 'j': int, 'k': int}")
    print("   - Przykład A: {'oper': 'A', 'i': 1, 'k': 2}")
    print("   - Przykład B: {'oper': 'B', 'i': 1, 'j': 2, 'k': 3}")
    print("   - Przykład C: {'oper': 'C', 'i': 1, 'j': 2, 'k': 3}")
    
    print("\n3. POSTAĆ NORMALNA FOATY (FNF):")
    print("   - Typ: Lista list operacji")
    print("   - Każda lista = klasa Foaty (operacje niezależne)")
    print("   - Struktura: [[op1, op2, ...], [op3, op4, ...], ...]")
    
    print("\n4. RELACJA ZALEŻNOŚCI (D):")
    print("   - Typ: Lista tupli (op1, op2)")
    print("   - Reprezentuje pary operacji zależnych")
    print("   - Wykorzystywana do budowy grafu")
    
    # Przykład użycia pamięci
    import sys
    
    print("\n5. ZUŻYCIE PAMIĘCI (szacunkowe dla N=3):")
    example_op = {"oper": "A", "i": 1, "k": 2}
    print(f"   - Jedna operacja: ~{sys.getsizeof(example_op)} bajtów")
    print(f"   - Alfabet ({len(alphabet)} operacji): ~{sys.getsizeof(alphabet) + len(alphabet) * sys.getsizeof(example_op)} bajtów")
    print(f"   - FNF ({len(fnf)} klas): ~{sys.getsizeof(fnf)} bajtów")

analyze_data_structures()

################################################################################
# Komórka 23: Przykład z Większą Macierzą
################################################################################

print("PRZYKŁAD: WIĘKSZY UKŁAD RÓWNAŃ (4x4)")
print("=" * 80)

# Definicja większego układu
N_big = 4
matrix_big = [
    [10.0, -1.0, 2.0, 0.0],
    [-1.0, 11.0, -1.0, 3.0],
    [2.0, -1.0, 10.0, -1.0],
    [0.0, 3.0, -1.0, 8.0]
]
column_big = [[6.0], [25.0], [-11.0], [15.0]]

complemented_big = [row + col for row, col in zip(matrix_big, column_big)]

print("\nMacierz początkowa [A|b]:")
for row in complemented_big:
    print([f"{x:7.2f}" for x in row])

# Generowanie struktur
alphabet_big, _ = get_alphabet(N_big)
fnf_big = get_fnf(N_big)
dependent_big, _ = get_dependent_transactions(N_big)

print(f"\nStatystyki:")
print(f"  - Rozmiar: {N_big}x{N_big}")
print(f"  - Operacji w alfabecie: {len(alphabet_big)}")
print(f"  - Klas Foaty: {len(fnf_big)}")
print(f"  - Par zależnych: {len(dependent_big)}")

# Wykonanie eliminacji
complemented_big_copy = copy.deepcopy(complemented_big)
eliminated_big, ops_big = parallel_gauss(complemented_big_copy, fnf_big)

print("\nMacierz po eliminacji:")
for row in eliminated_big:
    print([f"{x:7.3f}" for x in row])

# Podstawianie wsteczne
result_big_matrix = copy.deepcopy(eliminated_big)
results_big = backward_substitution(result_big_matrix, N_big)

print(f"\nRozwiązanie:")
for i, val in enumerate(results_big):
    print(f"  x_{i+1} = {val:8.4f}")

# Weryfikacja
print(f"\nWeryfikacja:")
for i in range(N_big):
    sum_val = sum(matrix_big[i][j] * results_big[j] for j in range(N_big))
    expected = column_big[i][0]
    diff = abs(sum_val - expected)
    print(f"  Równanie {i+1}: Ax = {sum_val:.4f}, b = {expected:.4f}, błąd = {diff:.6f}")

################################################################################
# Komórka 24: Eksport Grafu do Różnych Formatów
################################################################################

def export_graph_data(dependent, fnf, filename_base="gauss_graph"):
    """Eksportuje dane grafu do różnych formatów"""
    
    # Tworzenie grafu
    G = nx.DiGraph()
    
    # Dodawanie wierzchołków z atrybutami
    node_colors = {}
    for i, foata_class in enumerate(fnf):
        for op in foata_class:
            node_name = print_op(op)
            G.add_node(node_name, 
                      foata_class=i+1,
                      operation_type=op["oper"],
                      indices=f"({op.get('i', '')},{op.get('j', '')},{op.get('k', '')})")
            node_colors[node_name] = i
    
    # Dodawanie krawędzi
    for dep1, dep2 in dependent:
        node1 = print_op(dep1)
        node2 = print_op(dep2)
        if node1 in G.nodes() and node2 in G.nodes():
            G.add_edge(node1, node2)
    
    print("EKSPORT DANYCH GRAFU")
    print("=" * 80)
    
    # 1. GraphML (XML format)
    try:
        nx.write_graphml(G, f"{filename_base}.graphml")
        print(f"✓ Zapisano GraphML: {filename_base}.graphml")
    except Exception as e:
        print(f"✗ Błąd GraphML: {e}")
    
    # 2. GML
    try:
        nx.write_gml(G, f"{filename_base}.gml")
        print(f"✓ Zapisano GML: {filename_base}.gml")
    except Exception as e:
        print(f"✗ Błąd GML: {e}")
    
    # 3. Edgelist
    try:
        nx.write_edgelist(G, f"{filename_base}.edgelist")
        print(f"✓ Zapisano Edgelist: {filename_base}.edgelist")
    except Exception as e:
        print(f"✗ Błąd Edgelist: {e}")
    
    # 4. JSON
    try:
        import json
        data = nx.node_link_data(G)
        with open(f"{filename_base}.json", 'w') as f:
            json.dump(data, f, indent=2)
        print(f"✓ Zapisano JSON: {filename_base}.json")
    except Exception as e:
        print(f"✗ Błąd JSON: {e}")
    
    # Statystyki grafu
    print(f"\nStatystyki grafu:")
    print(f"  - Wierzchołki: {G.number_of_nodes()}")
    print(f"  - Krawędzie: {G.number_of_edges()}")
    print(f"  - Gęstość: {nx.density(G):.4f}")
    print(f"  - Czy acykliczny (DAG): {nx.is_directed_acyclic_graph(G)}")
    
    if nx.is_directed_acyclic_graph(G):
        print(f"  - Sortowanie topologiczne możliwe: TAK")
    
    return G

# Eksport (ustaw save=True aby zapisać)
save_files = False
if save_files:
    graph = export_graph_data(dependent, fnf)
else:
    print("ℹ Ustaw save_files=True aby zapisać pliki grafu")

################################################################################
# Komórka 25: Podsumowanie i Dokumentacja
################################################################################

def generate_summary(N, matrix, results, stats):
    """Generuje kompletne podsumowanie wyników"""
    
    print("\n" + "=" * 80)
    print(" " * 30 + "PODSUMOWANIE ANALIZY")
    print("=" * 80)
    
    print(f"\n📊 1. DANE WEJŚCIOWE:")
    print(f"   ├─ Rozmiar układu: {N}x{N}")
    print(f"   ├─ Macierz współczynników A: {N}x{N}")
    print(f"   └─ Wektor wyrazów wolnych b: {N}x1")
    
    print(f"\n🔬 2. ANALIZA TEORETYCZNA (TEORIA ŚLADÓW):")
    print(f"   ├─ Alfabet Σ: {stats['total_ops']} operacji")
    print(f"   │  ├─ Operacje A (obliczanie mnożników): {(N*(N-1))//2}")
    print(f"   │  ├─ Operacje B (mnożenie): {(N*(N-1)*(N+2))//2}")
    print(f"   │  └─ Operacje C (odejmowanie): {(N*(N-1)*(N+2))//2}")
    print(f"   ├─ Postać normalna Foaty (FNF): {stats['sequential_steps']} klas")
    print(f"   ├─ Maksymalne równoległość: {stats['max_parallel']} operacji")
    print(f"   └─ Teoretyczne przyspieszenie: {stats['speedup']:.2f}x")
    
    print(f"\n⚡ 3. WYKONANIE RÓWNOLEGŁE:")
    print(f"   ├─ Kroki sekwencyjne: {stats['sequential_steps']}")
    print(f"   ├─ Średnio operacji na krok: {stats['total_ops']/stats['sequential_steps']:.1f}")
    print(f"   └─ Efektywność równoległości: {(stats['speedup']/stats['max_parallel']*100):.1f}%")
    
    print(f"\n✅ 4. ROZWIĄZANIE UKŁADU:")
    print(f"   Wektor x = [", end="")
    print(", ".join([f"{r:.6f}" for r in results]), end="")
    print("]ᵀ")
    
    print(f"\n🔍 5. WERYFIKACJA:")
    verification = []
    for i in range(N):
        sum_val = sum(matrix[i][j] * results[j] for j in range(N))
        verification.append(sum_val)
    
    max_error = max(abs(verification[i] - column[i][0]) for i in range(N))
    print(f"   ├─ Maksymalny błąd numeryczny: {max_error:.2e}")
    print(f"   └─ Status: {'✅ POPRAWNE' if max_error < 1e-6 else '⚠️  WYMAGA SPRAWDZENIA'}")
    
    print(f"\n📈 6. ZŁOŻONOŚĆ OBLICZENIOWA:")
    classic_complexity = (N**3) / 3
    print(f"   ├─ Klasyczna eliminacja O(n³/3): ~{classic_complexity:.0f} operacji")
    print(f"   ├─ Nasza implementacja: {stats['total_ops']} operacji")
    print(f"   ├─ Czas sekwencyjny: O(n³)")
    print(f"   └─ Czas równoległy: O(n²) przy nieograniczonej liczbie procesorów")
    
    print("\n" + "=" * 80)
    print(" " * 25 + "✨ Analiza zakończona pomyślnie! ✨")
    print("=" * 80 + "\n")
    
    # Dodatkowe informacje
    print("💡 WNIOSKI:")
    print("   • Teoria śladów pozwala na automatyczną identyfikację niezależnych operacji")
    print("   • Postać normalna Foaty (FNF) wyznacza optymalny harmonogram równoległy")
    print("   • Graf Diekerta wizualizuje zależności między operacjami")
    print(f"   • Dla macierzy {N}x{N} możliwe jest przyspieszenie do {stats['speedup']:.1f}x")

generate_summary(N, matrix, results, stats)

# Opcjonalnie: zapis do pliku
def save_results_to_file(N, matrix, column, results, filename='wyniki_eliminacji.txt'):
    """Zapisuje wyniki do pliku tekstowego"""
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("╔" + "═" * 78 + "╗\n")
        f.write("║" + " " * 20 + "WYNIKI ELIMINACJI GAUSSA" + " " * 34 + "║\n")
        f.write("╚" + "═" * 78 + "╝\n\n")
        
        f.write(f"Rozmiar układu: {N}x{N}\n\n")
        
        f.write("Macierz początkowa [A|b]:\n")
        comp = [matrix[i] + [column[i][0]] for i in range(N)]
        for row in comp:
            f.write("  " + "  ".join([f"{x:8.3f}" for x in row]) + "\n")
        
        f.write("\n" + "-" * 80 + "\n\n")
        f.write("ROZWIĄZANIE:\n")
        for i, val in enumerate(results):
            f.write(f"  x_{i+1} = {val:.10f}\n")
        
        f.write("\n" + "-" * 80 + "\n\n")
        f.write("WERYFIKACJA (A·x = b):\n")
        for i in range(N):
            sum_val = sum(matrix[i][j] * results[j] for j in range(N))
            expected = column[i][0]
            f.write(f"  Równanie {i+1}: {sum_val:.6f} ≈ {expected:.6f} ")
            f.write(f"(błąd: {abs(sum_val - expected):.2e})\n")
    
    print(f"✓ Wyniki zapisane do pliku '{filename}'")

# Odkomentuj poniższą linię aby zapisać wyniki
# save_results_to_file(N, matrix, column, results)

################################################################################